# <center> Homework 3: Hidden Markov Model (HMM) </center>
<div>
    <center> RAZAFINDRAZAKA Henintsoa </center>
    <center> WANG James </center>
    <center> ____________</center>
    <center> ING5 SI GR02 </center>
    <center> 30/10/2020 </center>
</div>

# <center> [Tasks] </center>


> 1. Implement yourself the HMM algorithm to solve the evaluation and decoding problems for the `Example` given in class.
2. Compare your results to those that were given in this example.

## 1. Evaluation problem 

Suppose that a hidden state can take the following values: $z_0, z_1, z_2, z_3$.

Suppose also that an observable state can take values: $x_0, x_1, x_2, x_3, x_4$.

Then, assume also that both the transition and emission probabilities are given:

$$
\{a_{ij} \}
=
\begin{bmatrix}
1 & 0 & 0 & 0
\\ 
0.2 & 0.3 & 0.1 & 0.4 
\\
0.2 & 0.5 & 0.2 & 0.1
\\
0.7 & 0.1 & 0.1 & 0.1
\end{bmatrix}
$$

$$
\{b_{jk} \}
=
\begin{bmatrix}
1 & 0 & 0 & 0 & 0
\\ 
0 & 0.3 & 0.4 & 0.1 & 0.2
\\
0 & 0.1 & 0.1 & 0.7 & 0.1
\\
0 & 0.5 & 0.2 & 0.1 & 0.2
\end{bmatrix}
$$

Now, let us proceed to compute the probability of a given sequence of visible states from an HMM M.

Suppose that the following information is given: $x = \{ x_1, x_3, x_2, x_0 \}$ , $z(0) = z_1$, with transition and emission probabilities $a_{ij}$ and $b_{jk}$ given above.

Then, let us compute $P(x|M)$ using the HMM forward algorithm introduced in class.

### 1.1. Initialization of $\tilde{X}, \tilde{Z}, a_{ij}, b_{jk}, x \text{  and  } z(0)$

In [1]:
import numpy as np

N = 4 # total nb of possible values that a hidden state can take
z_set = []
for i in range(N):
    z_set.append( 'z'+str(i) )
print('z set of values = ',z_set)

K = 5 # total nb of possible values that an observable state can take
x_set = []
for i in range(K):
    x_set.append( 'x'+str(i) )
print('x set of values = ',x_set)

x = [x_set[k] for k in [1,3,2,0] ]
print('x = ',x)

T = len(x)

z set of values =  ['z0', 'z1', 'z2', 'z3']
x set of values =  ['x0', 'x1', 'x2', 'x3', 'x4']
x =  ['x1', 'x3', 'x2', 'x0']


In [2]:
a = np.asarray(
    [[1 ,  0,   0,   0],
    [0.2, 0.3, 0.1, 0.4],
    [0.2, 0.5, 0.2, 0.1],
    [0.7, 0.1, 0.1, 0.1]])

b = np.asarray(
    [[1,  0,  0,   0,   0 ],
    [0, 0.3, 0.4, 0.1, 0.2],
    [0, 0.1, 0.1, 0.7, 0.1],
    [0, 0.5, 0.2, 0.1, 0.2]])

print("a's shape : ",a.shape, "\nb's shape : ", b.shape )

a's shape :  (4, 4) 
b's shape :  (4, 5)


In [3]:
z = np.full((T + 1,), 'undefined')
z[0] = z_set[1]
print('z = ',z)

z =  ['z1' 'undefined' 'undefined' 'undefined' 'undefined']


### 1.2. Forward algorithm
#### A. Initialization of $α_j$ for $t = 0$


$$
\alpha_j(t=0)=\left\{
            \begin{array}{ll}
              0, \quad \text{if} \quad z_j \neq \text{initial state} \\
              1, \quad \text{if} \quad z_j =  \text{initial state}
            \end{array}
          \right.
$$

In [4]:
alpha = np.zeros((N, T+1))

j_initial_state = int(z[0][1])
for j in range(N):
    if j == j_initial_state:
        alpha[j][0] = 1 

print('α = \n',alpha)

α = 
 [[0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


#### B. Computation of $α_j$ for $t = 1,\dots,T$

$$
\alpha_j(t)= b_{jk} \sum^N_{i=1} \alpha_i(t-1)a_{ij}
$$

In [5]:
for t in range(1, T+1):
    for j in range(N):
        k = int(x[t-1][1])
        
        Sum = 0
        for i in range(N):
            Sum += alpha[i][t-1] * a[i][j]
            
        alpha[j][t] = b[j][k] * Sum
print('α = \n',alpha)

α = 
 [[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.8218e-03]
 [1.0000e+00 9.0000e-02 5.2000e-03 5.1920e-03 0.0000e+00]
 [0.0000e+00 1.0000e-02 2.1700e-02 5.4300e-04 0.0000e+00]
 [0.0000e+00 2.0000e-01 5.7000e-03 9.6400e-04 0.0000e+00]]


In [6]:
P_xm = alpha[0][T]
print('P(X|M) = ', P_xm)

P(X|M) =  0.0018218000000000002


#### <center> Results </center> 
> We have the same results than in the example given in class with 

> $P(X|M) = 0.0018218$

## 2. Decoding problem

### 2.1. Initialization of $\tilde{X}, \tilde{Z}, a_{ij}, b_{jk}, x \text{  and  } z(0)$     (identical to 1.1)

In [7]:
import numpy as np

N = 4 # total nb of possible values that a hidden state can take
z_set = []
for i in range(N):
    z_set.append( 'z'+str(i) )
print('z set of values = ',z_set)

K = 5 # total nb of possible values that an observable state can take
x_set = []
for i in range(K):
    x_set.append( 'x'+str(i) )
print('x set of values = ',x_set)

x = [x_set[k] for k in [1,3,2,0] ]
print('x = ',x)

T = len(x)

z set of values =  ['z0', 'z1', 'z2', 'z3']
x set of values =  ['x0', 'x1', 'x2', 'x3', 'x4']
x =  ['x1', 'x3', 'x2', 'x0']


In [8]:
a = np.asarray(
    [[1 ,  0,   0,   0],
    [0.2, 0.3, 0.1, 0.4],
    [0.2, 0.5, 0.2, 0.1],
    [0.7, 0.1, 0.1, 0.1]])

b = np.asarray(
    [[1,  0,  0,   0,   0 ],
    [0, 0.3, 0.4, 0.1, 0.2],
    [0, 0.1, 0.1, 0.7, 0.1],
    [0, 0.5, 0.2, 0.1, 0.2]])

print("a's shape : ",a.shape, "\nb's shape : ", b.shape )

a's shape :  (4, 4) 
b's shape :  (4, 5)


In [9]:
z = np.full((T + 1,), 'undefined')
z[0] = z_set[1]
print('z = ',z)

z =  ['z1' 'undefined' 'undefined' 'undefined' 'undefined']


### 2.2. Algorithm for decoding problem
#### A. Initialization of $α_j$ for $t = 0$, and $Path$


$$
\alpha_j(t=0)=\left\{
            \begin{array}{ll}
              0, \quad \text{if} \quad z_j \neq \text{initial state} \\
              1, \quad \text{if} \quad z_j =  \text{initial state}
            \end{array}
          \right.
$$

In [10]:
alpha = np.zeros((N, T+1))

j_initial_state = int(z[0][1])
for j in range(N):
    if j == j_initial_state:
        alpha[j][0] = 1 

print('α = \n',alpha)

α = 
 [[0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [11]:
Path = []
Path.append(z[0])
print('Path = ', Path)

Path =  ['z1']


#### B. Computation of $α_j$ for $t = 1,\dots,T$, and construction of $Path$

In [12]:
for t in range(1, T+1):
    jmax = 0
    for j in range(N):
        k = int(x[t-1][1])
        
        Sum = 0
        for i in range(N):
            Sum += alpha[i][t-1] * a[i][j]
            
        alpha[j][t] = b[j][k] * Sum
        
        if alpha[jmax][t] < alpha[j][t]:
            jmax = j
    Path.append(z_set[jmax])

print('α = \n',alpha)
print('\n  x  = ',x)
print('\nPath = ',Path)

α = 
 [[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.8218e-03]
 [1.0000e+00 9.0000e-02 5.2000e-03 5.1920e-03 0.0000e+00]
 [0.0000e+00 1.0000e-02 2.1700e-02 5.4300e-04 0.0000e+00]
 [0.0000e+00 2.0000e-01 5.7000e-03 9.6400e-04 0.0000e+00]]

  x  =  ['x1', 'x3', 'x2', 'x0']

Path =  ['z1', 'z3', 'z2', 'z1', 'z0']


#### <center> Results </center> 
> As in the decoding example given in class, the most probable sequence of hidden states Z that would have generated the sequence X appears to be: 

> $Path = \{ z_1, z_3, z_2, z_1, z_0 \}$